In [ ]:
import sklearn
from sklearn.datasets import make_circles

n_samples = 1000
X, y = make_circles(n_samples, noise=0.03, random_state=42)
# print(X[:5])
# print(y[:5])

import pandas as pd
circles = pd.DataFrame({'X1': X[:, 0], 'X2': X[:, 1], 'label': y})
# circles.head(10)

import matplotlib.pyplot as plt
plt.scatter(x=X[:, 0], y=X[:, 1], c=y, cmap=plt.cm.RdYlBu)

print(circles.label.value_counts())

In [ ]:
import torch

print(X.dtype, type(X))
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)
print(X.dtype, type(X))

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train), len(X_test))

In [ ]:
from torch import nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class CircleModelV0(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.layer1 = nn.Linear(in_features=2, out_features=5)
        self.layer2 = nn.Linear(in_features=5, out_features=1)
        
    def forward(self, x):
        return self.layer2(self.layer1(x))

model0 = CircleModelV0().to(device)
print(model0.parameters)

model0 = nn.Sequential(
    nn.Linear(in_features=2, out_features=5),
    nn.Linear(in_features=5, out_features=1)
).to(device)
# print(model1.parameters)
# print(model1.state_dict())

loss_fn = nn.BCEWithLogitsLoss() # nn.Sigmoid() + BCELoss
optim = torch.optim.SGD(params=model0.parameters(), lr=0.1)

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct/len(y_pred)) * 100
    return acc

model0.eval()
with torch.inference_mode():
    y_logits = model0(X_test.to(device))[:5]
    y_preds = torch.round(torch.sigmoid(y_logits))
print(y_preds)

In [ ]:
torch.manual_seed(42)
# torch.cuda.manual_seed(42)

epochs = 100
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

for epoch in range(epochs):
    model0.train()
    
    y_logits = model0(X_train).squeeze()
    y_pred = torch.round(torch.sigmoid(y_logits))
    
    loss = loss_fn(y_logits, y_train) # BCEWithLogitsLoss requires logits as an input
    # loss = loss_fn(torch.sigmoid(y_logits), y_train) #this is loss if we choose BCELoss as our loss function as it requires prediction probs
    acc = accuracy_fn(y_train, y_pred)
    
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    model0.eval()
    with torch.inference_mode():
        test_logits = model0(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))
        
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_test, test_pred)
    if epoch % 10 == 0:
        print('Epoch:', epoch, 'train loss:', loss, 'train acc:', acc, 'test loss:', test_loss, 'test acc:', test_acc)

In [ ]:
import requests
from pathlib import Path

if Path('helper_functions.py').is_file():
    print('Already exists')
else:
    print('Downloading')
    request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
    with open('helper_functions.py', 'wb') as f:
        f.write(request.content)
    print('Downloading finished')
from helper_functions import plot_predictions, plot_decision_boundary

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Train')
plot_decision_boundary(model0, X_train, y_train)
plt.subplot(1, 2, 2)
plt.title('Test')
plot_decision_boundary(model0, X_test, y_test)

#### To improve a model's accuracy:
    - Add more layers
    - Add more hidden units
    - Fit for more epochs
    - Change activation function
    - Change the learning rate
    - Change the loss function

In [ ]:
class CircleModelV1(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(in_features=2, out_features=10)
        self.layer2 = nn.Linear(in_features=10, out_features=10)
        self.layer3 = nn.Linear(in_features=10, out_features=1)
    
    def forward(self, x):
        return self.layer3(self.layer2(self.layer1(x))) # this style speed up the calculation behind the scenes
    
model1 = CircleModelV1().to(device)

loss_fn = nn.BCEWithLogitsLoss()
optim = torch.optim.SGD(params=model1.parameters(), lr=0.1)

In [ ]:
torch.manual_seed(42)
# torch.cuda.manual_seed(42)

epochs = 1000
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

for epoch in range(epochs):
    model1.train()
    
    y_logits = model1(X_train).squeeze()
    y_pred = torch.round(torch.sigmoid(y_logits))
    
    loss = loss_fn(y_logits, y_train)
    acc = accuracy_fn(y_train, y_pred)
    
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    model1.eval()
    with torch.inference_mode():
        test_logits = model1(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))
        
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_test, test_pred)
        
    if epoch%100==0:
        print(f"Epoch: {epoch}, Loss: {loss:.5f}, Acc: {acc:.2f}, Test_loss: {test_loss:.5f}, Test acc: {test_acc:.2f}")

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Train')
plot_decision_boundary(model1, X_train, y_train)
plt.subplot(1, 2, 2)
plt.title('Test')
plot_decision_boundary(model1, X_test, y_test)

### One way to troubleshoot to a larger problem - test out a smaller problem

In [ ]:
weight = 0.7
bias = 0.3
start = 0
end = 1
step = 0.01

X_regression = torch.arange(start, end, step).unsqueeze(dim=1)
y_regression = weight * X_regression + bias
# print(len(X_regression), len(y_regression))

train_split = int(0.8*len(X_regression))
X_train_reg, y_train_reg = X_regression[:train_split], y_regression[:train_split]
X_test_reg, y_test_reg = X_regression[train_split:], y_regression[train_split:]
# print(len(X_train_reg), len(y_train_reg))

# plot_predictions(X_train_reg, y_train_reg, X_test_reg, y_test_reg)

model2 = nn.Sequential(
    nn.Linear(in_features=1, out_features=10),
    nn.Linear(in_features=10, out_features=10),
    nn.Linear(in_features=10, out_features=1),
).to(device)

torch.manual_seed(42)
# torch.cuda.manual_seed(42)
loss_fn = nn.L1Loss()
optim = torch.optim.SGD(params=model2.parameters(),
                       lr=0.005)
epochs = 1000
X_train_reg, y_train_reg = X_train_reg.to(device), y_train_reg.to(device)
X_test_reg, y_test_reg = X_test_reg.to(device), y_test_reg.to(device)

for epoch in range(epochs):
    y_pred = model2(X_train_reg)
    loss = loss_fn(y_pred, y_train_reg)
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    model2.eval()
    with torch.inference_mode():
        test_pred = model2(X_test_reg)
        test_loss = loss_fn(test_pred, y_test_reg)
        
    if epoch%100==0:
        print(f"Epoch: {epoch} Loss: {loss}, Test loss: {test_loss}")

In [ ]:
model2.eval()
with torch.inference_mode():
#     train_pred = model2(X_train_reg)
    test_pred = model2(X_test_reg).to(device)
plot_predictions(train_data=X_train_reg, 
                 train_labels=y_train_reg, 
                 test_data=X_test_reg, 
                 test_labels=y_test_reg, 
                 predictions=test_pred)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles

n_samples = 1000
X, y = make_circles(n_samples, noise=0.03, random_state=42)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu)

In [ ]:
import torch
from sklearn.model_selection import train_test_split

X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train[:5], y_train[:5])

In [ ]:
class CircleModelV2(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layer1 = nn.Linear(in_features=2, out_features=10)
        self.layer2 = nn.Linear(in_features=10, out_features=10)
        self.layer3 = nn.Linear(in_features=10, out_features=1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        return self.layer3(self.relu(self.layer2(self.relu(self.layer1(x)))))
    
model3 = CircleModelV2().to(device)
# print(model2.state_dict())
optim = torch.optim.SGD(params=model3.parameters(), lr=0.1)
loss_fn = nn.BCEWithLogitsLoss()

torch.manual_seed(42)
torch.cuda.manual_seed(42)
X_train, X_test, y_train, y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)
epochs = 1500

for epoch in range(epochs):
    model3.train()
    y_logits = model3(X_train).squeeze()
    y_pred = torch.round(torch.sigmoid(y_logits))
    
    loss = loss_fn(y_logits, y_train)
    train_acc = accuracy_fn(y_train, y_pred)
    
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    model3.eval()
    with torch.inference_mode():
        test_logits = model3(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))
        
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_test, test_pred)
    
    if epoch%150==0:
        print(f"Epoch: {epoch} Loss: {loss:.5f} Accuracy: {train_acc:.2f}% Test Loss: {test_loss:.5f} Test Accuracy: {test_acc:.2f}%")

In [ ]:
model3.eval()
with torch.inference_mode():
    model3.eval()
    pred = torch.round(torch.sigmoid(model3(X_test))).squeeze().to(device)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Train')
plot_decision_boundary(model3, X_train, y_train)
plt.subplot(1, 2, 2)
plt.title('Test')
plot_decision_boundary(model3, X_test, y_test)

### Multi class classicition

In [ ]:
import torch
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

NUM_CLASSES = 4
NUM_FEATURES = 2
RANDOM_SEED = 42

X, y = make_blobs(n_samples=1000,
                 n_features=NUM_FEATURES,
                 centers=NUM_CLASSES,
                 cluster_std=1.5,
                 random_state=RANDOM_SEED)
X, y = torch.from_numpy(X).type(torch.float), torch.from_numpy(y).type(torch.LongTensor)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
# print(len(X_train), len(X_test))

plt.figure(figsize=(10, 7))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class BlobsModelV1(nn.Module):
    def __init__(self, input_features, output_features, hidden_units=8):
        super().__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_features),
            nn.Softmax()
        )
        
    def forward(self, x):
        return self.linear_stack(x)
    
model1 = BlobsModelV1(input_features=NUM_FEATURES, output_features=NUM_CLASSES).to(device)
# print(model1.state_dict())
optim = torch.optim.SGD(params=model1.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
epochs = 100

X_train, X_test, y_train, y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

for epoch in range(epochs):
    model1.train()
    y_preds = model1(X_train)
    y_labels = torch.argmax(y_preds, dim=1)

    loss = loss_fn(y_preds, y_train)
    acc = accuracy_fn(y_train, y_labels)
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    model1.eval()
    with torch.inference_mode():
        test_preds = model1(X_test)
        test_labels = torch.argmax(test_preds, dim=1)
        
        test_loss = loss_fn(test_preds, y_test)
        test_acc = accuracy_fn(y_test, test_labels)
        
    if epoch%10==0:
        print(f"Epoch: {epoch} Loss: {loss:.5f} Acc: {acc:.2f}% Test Loss: {test_loss:.5f} Test Acc: {test_acc:.2f}%")

In [ ]:
model1.eval()
with torch.inference_mode():
    y_preds = model1(X_test)
    y_labels = torch.argmax(y_preds, dim=1)
    
print(y_labels[:10])

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Train')
plot_decision_boundary(model1, X_train, y_train)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 2)
plt.title('Test')
plot_decision_boundary(model1, X_test, y_test)

In [ ]:
import torchmetrics

# print(y_labels.dtype, y_test.dtype)
acc = torchmetrics.Accuracy(task='multiclass', num_classes=4)(y_labels, y_test)
# print(acc)
acc = accuracy_fn(y_test, y_labels)
# print(acc)

from torchmetrics.classification import MulticlassConfusionMatrix
metric = MulticlassConfusionMatrix(num_classes=4)
metric.update(y_test, y_labels)
# fig_, ax_ = metric.plot()

from torchmetrics.classification import MulticlassPrecisionRecallCurve
metric = MulticlassPrecisionRecallCurve(num_classes=4)
print(y_preds.shape, y_labels.shape)
metric.update(y_preds, y_labels)
fig_, ax_ = metric.plot(score=True)